# Soccer Analytics - Data Exploration Notebook

This notebook loads the StatsBomb Parquet extracts, inspects the main tables (matches, events, lineups, three-sixty, reference), and performs basic data-quality checks and small exploratory analyses. Use the sections below to navigate the notebook.

Sections:
- Setup and imports
- Data loading and preview (matches, events, lineups, three-sixty, reference)
- Quick checks (date range, coverage, orphaned records)
- Small examples (freeze frame, shots)
- Splitting dataset for modeling

Note: run cells top-to-bottom to ensure required variables are available.

In [ ]:
# Setup and imports
# This cell installs small deps (if needed), imports libraries and sets display options.
!pip install matplotlib && pip install -q pandas
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import warnings
import seaborn as sns
from pathlib import Path

# Suppress common warnings for cleaner notebook output
warnings.filterwarnings("ignore")

# Display options for notebooks
pd.set_option("display.max_columns", None)
pd.set_option("display.width", 120)

# Data directory
DATA_DIR = Path("..") / "data" / "Statsbomb"
print(f"DATA_DIR: {DATA_DIR}")
print('Files present:', sorted([p.name for p in DATA_DIR.iterdir() if p.is_file()]))


[notice] A new release of pip is available: 23.2.1 -> 25.3
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 25.3
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 25.3
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 25.3
[notice] To update, run: pip install --upgrade pip
DATA_DIR: ../data/Statsbomb
Files present: ['events.parquet', 'lineups.parquet', 'matches.parquet', 'reference.parquet', 'statsbomb_schema.md', 'three_sixty.parquet']
DATA_DIR: ../data/Statsbomb
Files present: ['events.parquet', 'lineups.parquet', 'matches.parquet', 'reference.parquet', 'statsbomb_schema.md', 'three_sixty.parquet']


In [90]:
# Matches: one row per match
matches_path = DATA_DIR / "matches.parquet"
if not matches_path.exists():
    raise FileNotFoundError(f"Missing {matches_path}. Make sure data is downloaded into data/Statsbomb")

matches = pd.read_parquet(matches_path)
# Show a compact preview to confirm successful load
print('Matches shape:', matches.shape)
matches.head(3)

Matches shape: (3464, 36)


,match_id,match_date,match_week,match_status,match_status_360,kickoff,home_score,away_score,competition_id,competition,competition_stage,season_id,season,home_team_id,home_team,home_managers,away_team_id,away_team,away_managers,stadium_id,stadium,referee_id,referee,last_updated,last_updated_360,data_version,shot_fidelity_version,xy_fidelity_version,competition_name,gender,is_youth,is_international,country_name,season_name,match_updated,match_available_360
0,9880,2018-04-14,32,available,scheduled,16:15:00,2,1,11,La Liga,Regular Season,1,2017/2018,217,Barcelona,"[{""id"":227,""name"":""Ernesto Valverde Tejedor"",""...",207,Valencia,"[{""id"":211,""name"":""Marcelino García Toral"",""ni...",342.0,Spotify Camp Nou,2728.0,Carlos del Cerro Grande,2023-02-08T17:23:53.901920,2021-06-13T16:17:31.694,1.1.0,2,2,La Liga,male,False,False,Spain,2017/2018,2025-07-14T10:01:16.674864,NaN
1,9912,2018-04-29,35,available,scheduled,20:45:00,2,4,11,La Liga,Regular Season,1,2017/2018,219,RC Deportivo La Coruña,"[{""id"":371,""name"":""Clarence Seedorf"",""nickname...",217,Barcelona,"[{""id"":227,""name"":""Ernesto Valverde Tejedor"",""...",4658.0,Estadio Abanca-Riazor,2602.0,Ricardo De Burgos Bengoetxea,2022-12-05T14:42:44.641092,2021-06-13T16:17:31.694,1.1.0,2,2,La Liga,male,False,False,Spain,2017/2018,2025-07-14T10:01:16.674864,NaN
2,9924,2018-05-06,36,available,scheduled,20:45:00,2,2,11,La Liga,Regular Season,1,2017/2018,217,Barcelona,"[{""id"":227,""name"":""Ernesto Valverde Tejedor"",""...",220,Real Madrid,"[{""id"":56,""name"":""Zinédine Zidane"",""nickname"":...",342.0,Spotify Camp Nou,2608.0,Alejandro José Hernández Hernández,2022-12-01T03:25:12.063586,2021-06-13T16:17:31.694,1.1.0,2,2,La Liga,male,False,False,Spain,2017/2018,2025-07-14T10:01:16.674864,NaN


In [121]:
# --- Matches: Enhanced Deep EDA ---
print('\n' + '='*80)
print('DEEP DIVE: matches (Enhanced)')
print('='*80)

# 1) Basic counts and date range
n_matches = len(matches)
print(f'Total matches: {n_matches:,}')

# Date analysis
matches['match_date'] = pd.to_datetime(matches['match_date'], errors='coerce')
min_date = matches['match_date'].min()
max_date = matches['match_date'].max()
date_span_days = (max_date - min_date).days
print(f'Date range: {min_date} → {max_date} ({date_span_days:,} days)')

# Temporal distribution
matches['year'] = matches['match_date'].dt.year
matches['month'] = matches['match_date'].dt.month
matches['day_of_week'] = matches['match_date'].dt.day_name()

print('\nMatches by year:')
year_dist = matches['year'].value_counts().sort_index()
for yr, cnt in year_dist.items():
    print(f'  {yr}: {cnt:4d} matches')

print('\nMatches by day of week:')
dow_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
dow_dist = matches['day_of_week'].value_counts().reindex(dow_order, fill_value=0)
for day, cnt in dow_dist.items():
    pct = cnt/n_matches*100
    print(f'  {day:9s}: {cnt:4d} ({pct:5.2f}%)')

# Detect gaps in coverage
match_dates = matches['match_date'].dt.date.value_counts().sort_index()
date_diffs = pd.Series(match_dates.index).diff().dt.days.dropna()
long_gaps = date_diffs[date_diffs > 30]
if not long_gaps.empty:
    print(f'\nLong gaps in coverage (>30 days): {len(long_gaps)} instances')
    print(f'  Longest gap: {date_diffs.max():.0f} days')

# 2) Competition Analysis
comp_counts = matches['competition_name'].fillna('MISSING').value_counts(dropna=False)
print('\n--- Competition Coverage ---')
print(f'Total competitions: {len(comp_counts)}')
print('\nTop 15 competitions:')
for i, (comp, cnt) in enumerate(comp_counts.head(15).items(), 1):
    pct = cnt/n_matches*100
    print(f'{i:2d}. {comp:40s}: {cnt:4d} ({pct:5.2f}%)')

# Competition-Season coverage
print('\n--- Competition × Season Coverage ---')
comp_season_counts = matches.groupby(['competition_name', 'season_name']).size().reset_index(name='matches')

# For each top competition, show season coverage
top_comps = comp_counts.head(5).index
for comp in top_comps:
    comp_data = comp_season_counts[comp_season_counts['competition_name'] == comp]
    n_seasons = len(comp_data)
    avg_matches = comp_data['matches'].mean()
    std_matches = comp_data['matches'].std()
    
    print(f'\n{comp}:')
    print(f'  Seasons covered: {n_seasons}')
    print(f'  Avg matches/season: {avg_matches:.1f} (±{std_matches:.1f})')
    
    # Flag outlier seasons
    if n_seasons > 2:
        q1 = comp_data['matches'].quantile(0.25)
        outliers = comp_data[comp_data['matches'] < q1 * 0.5]
        if not outliers.empty:
            print(f'  ⚠ Potential incomplete seasons ({len(outliers)}):')
            for _, row in outliers.head(3).iterrows():
                print(f'    - {row["season_name"]}: {row["matches"]} matches')

# 3) Team Analysis
all_teams = pd.concat([matches['home_team'], matches['away_team']])
team_counts = all_teams.value_counts()

print(f'\n--- Team Coverage ---')
print(f'Total unique teams: {len(team_counts)}')
print(f'Most frequent team: {team_counts.index[0]} ({team_counts.iloc[0]} appearances)')
print(f'Least frequent team: {team_counts.index[-1]} ({team_counts.iloc[-1]} appearances)')

# Teams appearing in multiple competitions
team_comp_counts = matches.melt(
    id_vars=['competition_name'], 
    value_vars=['home_team', 'away_team'], 
    value_name='team'
).groupby('team')['competition_name'].nunique().sort_values(ascending=False)

multi_comp_teams = team_comp_counts[team_comp_counts > 1]
print(f'\nTeams in multiple competitions: {len(multi_comp_teams)}')
if not multi_comp_teams.empty:
    print('Top 5 multi-competition teams:')
    for team, n_comps in multi_comp_teams.head(5).items():
        print(f'  {team}: {n_comps} competitions')

# 4) Score Analysis
print(f"\n--- Match Outcomes ---")

# Basic outcome distribution
home_wins = (matches['home_score'] > matches['away_score']).sum()
away_wins = (matches['away_score'] > matches['home_score']).sum()
draws = (matches['home_score'] == matches['away_score']).sum()

print('Results:')
print(f'  Home wins: {home_wins:5d} ({home_wins/n_matches*100:5.2f}%)')
print(f'  Draws:     {draws:5d} ({draws/n_matches*100:5.2f}%)')
print(f'  Away wins: {away_wins:5d} ({away_wins/n_matches*100:5.2f}%)')

# Goal statistics
total_home_goals = matches['home_score'].sum()
total_away_goals = matches['away_score'].sum()
total_goals = total_home_goals + total_away_goals

print(f'\nGoal statistics:')
print(f'  Total goals: {total_goals:.0f}')
print(f'  Avg goals/match: {total_goals/n_matches:.2f}')
print(f'  Avg home goals: {total_home_goals/n_matches:.2f}')
print(f'  Avg away goals: {total_away_goals/n_matches:.2f}')

# Scoreline distribution
matches['scoreline'] = matches.apply(
    lambda r: f"{int(r['home_score'])}-{int(r['away_score'])}", axis=1
)
scoreline_dist = matches['scoreline'].value_counts()

print('\nMost common scorelines:')
for i, (score, cnt) in enumerate(scoreline_dist.head(10).items(), 1):
    pct = cnt/n_matches*100
    print(f'{i:2d}. {score:6s}: {cnt:4d} ({pct:5.2f}%)')

# High/low scoring matches
matches['total_goals'] = matches['home_score'] + matches['away_score']
high_scoring = matches[matches['total_goals'] >= 6]
low_scoring = matches[matches['total_goals'] == 0]

print(f'\nHigh-scoring matches (6+ goals): {len(high_scoring)} ({len(high_scoring)/n_matches*100:.2f}%)')
print(f'Goalless draws (0-0): {len(low_scoring)} ({len(low_scoring)/n_matches*100:.2f}%)')

# Goal difference distribution
matches['goal_diff'] = matches['home_score'] - matches['away_score']
gd_dist = matches['goal_diff'].value_counts().sort_index()

print('\nGoal difference distribution (home - away):')
for gd in range(-5, 6):
    if gd in gd_dist.index:
        cnt = gd_dist[gd]
        pct = cnt/n_matches*100
        bar = '█' * int(pct/2)
        print(f'  {gd:+3d}: {cnt:4d} ({pct:5.2f}%) {bar}')

# Competition-specific home advantage
print('\nHome win rate by competition (top 5):')
comp_home_wr = matches.groupby('competition_name').apply(
    lambda x: (x['home_score'] > x['away_score']).sum() / len(x) * 100
).sort_values(ascending=False)

for comp, wr in comp_home_wr.head(5).items():
    comp_matches = matches[matches['competition_name'] == comp]
    print(f'  {comp:40s}: {wr:5.2f}% ({len(comp_matches)} matches)')

# 5) Data Quality Assessment
print('\n--- Data Quality ---')

# Missing values
miss = matches.isnull().sum()
miss_pct = (miss / len(matches) * 100).round(2)
miss_df = pd.DataFrame({'count': miss, 'pct': miss_pct})
miss_df = miss_df[miss_df['count'] > 0].sort_values('count', ascending=False)

if miss_df.empty:
    print('✓ No missing values in any column')
else:
    print(f'Columns with missing values: {len(miss_df)}/{len(matches.columns)}')
    print('\nTop 10 columns by missing data:')
    for col, row in miss_df.head(10).iterrows():
        print(f'  {col:40s}: {row["count"]:6.0f} ({row["pct"]:5.2f}%)')

# Duplicates by match_id
if 'match_id' in matches.columns:
    dup_id_count = matches['match_id'].duplicated().sum()
    print(f'\nDuplicate match_ids: {dup_id_count}')
    if dup_id_count > 0:
        print('⚠ Warning: Duplicate match_ids detected!')
        dup_examples = matches[matches['match_id'].duplicated(keep=False)].sort_values('match_id').head(3)
        print('Sample duplicates:')
        display(dup_examples[['match_id', 'home_team', 'away_team']])

# Duplicates by logical key (date, teams)
dup_logical = matches.duplicated(subset=['match_date', 'home_team', 'away_team']).sum()
print(f'Duplicate matches by (date, home_team, away_team): {dup_logical}')
if dup_logical > 0:
    print('⚠ Warning: Same teams playing on same date multiple times!')

# Data type summary
print('\nData type summary:')
dtype_counts = matches.dtypes.value_counts()
for dtype, cnt in dtype_counts.items():
    print(f'  {str(dtype):20s}: {cnt} columns')

# Unusual values check
max_home = matches['home_score'].max()
max_away = matches['away_score'].max()

if max_home > 10 or max_away > 10:
    print(f'\n⚠ Unusually high scores detected:')
    print(f'  Max home score: {max_home:.0f}')
    print(f'  Max away score: {max_away:.0f}')
    
    extreme = matches[(matches['home_score'] > 10) | (matches['away_score'] > 10)]
    if not extreme.empty:
        print(f'  Matches with 10+ goals for one team: {len(extreme)}')
        print('  Examples:')
        display(extreme[['match_date', 'home_team', 'away_team', 'home_score', 'away_score']].head(3))

print('\n' + '='*80)
print('Matches deep EDA complete.')
print('='*80)


DEEP DIVE: matches (Enhanced)
Total matches: 3,464
Date range: 1958-06-24 00:00:00 → 2025-07-27 00:00:00 (24,505 days)

Matches by year:
  1958:    2 matches
  1962:    1 matches
  1970:    6 matches
  1971:    1 matches
  1972:    1 matches
  1973:    1 matches
  1974:    7 matches
  1977:    1 matches
  1978:    1 matches
  1979:    1 matches
  1981:    1 matches
  1983:    1 matches
  1984:    1 matches
  1986:    4 matches
  1989:    3 matches
  1990:    1 matches
  1997:    1 matches
  1999:    1 matches
  2003:   19 matches
  2004:   25 matches
  2005:   13 matches
  2006:   16 matches
  2007:   33 matches
  2008:   27 matches
  2009:   31 matches
  2010:   37 matches
  2011:   36 matches
  2012:   39 matches
  2013:   27 matches
  2014:   37 matches
  2015:  893 matches
  2016:  967 matches
  2017:   39 matches
  2018:  194 matches
  2019:  190 matches
  2020:  120 matches
  2021:  204 matches
  2022:  193 matches
  2023:  105 matches
  2024:  153 matches
  2025:   31 matches



,match_date,home_team,away_team,home_score,away_score
2031,2019-12-01,Arsenal WFC,Bristol City WFC,11,1
3068,2019-06-11,United States Women's,Thailand Women's,13,0



Matches deep EDA complete.


In [124]:
# Events: one row per event per match 
events_path = DATA_DIR / "events.parquet"
if not events_path.exists():
    raise FileNotFoundError(f"Missing {events_path}. Make sure data is downloaded into data/Statsbomb")

events = pd.read_parquet(events_path)
# Show a compact preview to confirm successful load
print('Events shape:', events.shape)
events.head(3)

Events shape: (12188949, 112)


,id,index_num,period,minute,second,timestamp,duration,location_x,location_y,possession,possession_team_id,possession_team,out,off_camera,counterpress,under_pressure,type_id,type,match_id,team_id,team,player_id,player,position_id,position,play_pattern_id,play_pattern,shot_end_location_x,shot_end_location_y,shot_end_location_z,shot_statsbomb_xg,shot_outcome,shot_technique,shot_body_part,shot_type,shot_key_pass_id,shot_freeze_frame,shot_first_time,shot_deflected,shot_aerial_won,shot_follows_dribble,shot_one_on_one,shot_open_goal,shot_redirect,shot_saved_off_target,shot_saved_to_post,pass_end_location_x,pass_end_location_y,pass_recipient_id,pass_recipient,pass_length,pass_angle,pass_height,pass_body_part,pass_type,pass_outcome,pass_technique,pass_assisted_shot_id,pass_goal_assist,pass_shot_assist,pass_cross,pass_switch,pass_through_ball,pass_aerial_won,pass_deflected,pass_inswinging,pass_outswinging,pass_no_touch,pass_cut_back,pass_straight,pass_miscommunication,carry_end_location_x,carry_end_location_y,dribble_outcome,dribble_nutmeg,dribble_overrun,dribble_no_touch,duel_type,duel_outcome,foul_committed_card,foul_committed_type,foul_committed_offensive,foul_committed_advantage,foul_committed_penalty,foul_won_defensive,foul_won_advantage,foul_won_penalty,goalkeeper_type,goalkeeper_outcome,goalkeeper_technique,goalkeeper_position,goalkeeper_body_part,goalkeeper_end_location_x,goalkeeper_end_location_y,clearance_body_part,clearance_aerial_won,clearance_head,clearance_left_foot,clearance_right_foot,interception_outcome,block_deflection,block_offensive,block_save_block,ball_recovery_offensive,ball_recovery_failure,miscontrol_aerial_won,substitution_replacement_id,substitution_replacement_name,substitution_outcome,fifty_fifty_outcome,bad_behaviour_card,injury_stoppage_in_chain
0,9f6e2ecf-6685-45df-a62e-c2db3090f6c1,1,1,0,0,00:00:00,0.0,NaN,NaN,1,217,Barcelona,False,False,False,False,35,Starting XI,15946,217,Barcelona,NaN,NaN,NaN,NaN,1,Regular Play,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False,False,False,False,False,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False,False,False,False,False,False,False,False,False,False,False,NaN,NaN,NaN,False,False,False,NaN,NaN,NaN,NaN,False,False,False,False,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False,False,NaN,False,False,False,False,False,False,NaN,NaN,NaN,NaN,NaN,False
1,0300039d-150d-41e4-b29a-76602ef002e6,2,1,0,0,00:00:00,0.0,NaN,NaN,1,217,Barcelona,False,False,False,False,35,Starting XI,15946,206,Deportivo Alavés,NaN,NaN,NaN,NaN,1,Regular Play,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False,False,False,False,False,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False,False,False,False,False,False,False,False,False,False,False,NaN,NaN,NaN,False,False,False,NaN,NaN,NaN,NaN,False,False,False,False,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False,False,NaN,False,False,False,False,False,False,NaN,NaN,NaN,NaN,NaN,False
2,491e8901-7630-4cc8-b57b-937dddff2eaa,3,1,0,0,00:00:00,0.0,NaN,NaN,1,217,Barcelona,False,False,False,False,18,Half Start,15946,217,Barcelona,NaN,NaN,NaN,NaN,1,Regular Play,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False,False,False,False,False,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False,False,False,False,False,False,False,False,False,False,False,NaN,NaN,NaN,False,False,False,NaN,NaN,NaN,NaN,False,False,False,False,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False,False,NaN,False,False,False,False,False,False,NaN,NaN,NaN,NaN,NaN,False


In [ ]:
# --- Events: Deep EDA ---
print('\n' + '='*80)
print('DEEP DIVE: events')
print('='*80)

# 1) Basic Overview
n_events = len(events)
print(f'Total events: {n_events:,}')
print(f'Total columns: {len(events.columns)}')

# Match coverage
n_unique_matches = events['match_id'].nunique()
print(f'\nUnique matches: {n_unique_matches:,}')
print(f'Avg events per match: {n_events/n_unique_matches:.1f}')

# Events per match distribution
events_per_match = events.groupby('match_id').size()
print(f'\nEvents per match distribution:')
print(f'  Min: {events_per_match.min():,}')
print(f'  25th percentile: {events_per_match.quantile(0.25):,.0f}')
print(f'  Median: {events_per_match.median():,.0f}')
print(f'  75th percentile: {events_per_match.quantile(0.75):,.0f}')
print(f'  Max: {events_per_match.max():,}')

# Flag matches with unusually few events (potential data issues)
low_event_threshold = events_per_match.quantile(0.05)
low_event_matches = events_per_match[events_per_match < low_event_threshold]
if not low_event_matches.empty:
    print(f'\n⚠ Matches with unusually few events (<{low_event_threshold:.0f}): {len(low_event_matches)}')
    print('  Top 3 examples:')
    for match_id, count in low_event_matches.head(3).items():
        print(f'    match_id {match_id}: {count} events')

# 2) Event Type Distribution
print('\n--- Event Type Distribution ---')
event_type_counts = events['type'].fillna('MISSING').value_counts()
print(f'Unique event types: {len(event_type_counts)}')
print('\nTop 20 event types:')
for i, (event_type, cnt) in enumerate(event_type_counts.head(20).items(), 1):
    pct = cnt/n_events*100
    bar = '█' * int(pct/5)
    print(f'{i:2d}. {event_type:25s}: {cnt:8,} ({pct:5.2f}%) {bar}')

# Event type coverage across matches
event_type_match_coverage = events.groupby('type')['match_id'].nunique()
print('\nEvent type match coverage (top 10):')
for event_type in event_type_counts.head(10).index:
    matches_with_event = event_type_match_coverage[event_type]
    pct = matches_with_event/n_unique_matches*100
    print(f'  {event_type:25s}: {matches_with_event:5,}/{n_unique_matches:,} matches ({pct:5.2f}%)')

# 3) Team and Player Coverage
print('\n--- Team & Player Coverage ---')
n_teams = events['team'].nunique()
n_players = events['player'].nunique() if 'player' in events.columns else events['player_id'].nunique()
print(f'Unique teams: {n_teams}')
print(f'Unique players: {n_players:,}')

# Team activity
team_event_counts = events['team'].value_counts()
print(f'\nMost active teams (by events):')
for i, (team, cnt) in enumerate(team_event_counts.head(10).items(), 1):
    print(f'{i:2d}. {team:30s}: {cnt:8,} events')

# Player activity (excluding NaN)
if 'player' in events.columns:
    player_event_counts = events['player'].value_counts()
    print(f'\nMost active players (by events):')
    for i, (player, cnt) in enumerate(player_event_counts.head(10).items(), 1):
        print(f'{i:2d}. {player:30s}: {cnt:6,} events')
    
    # Events without player (e.g., team-level events)
    events_no_player = events['player'].isna().sum()
    print(f'\nEvents without player assignment: {events_no_player:,} ({events_no_player/n_events*100:.2f}%)')

# 4) Temporal Analysis
print('\n--- Temporal Distribution ---')

# Period analysis
if 'period' in events.columns:
    period_counts = events['period'].fillna(-1).value_counts().sort_index()
    print('Events by period:')
    for period, cnt in period_counts.items():
        pct = cnt/n_events*100
        period_label = f'Period {int(period)}' if period >= 0 else 'MISSING'
        print(f'  {period_label:12s}: {cnt:8,} ({pct:5.2f}%)')

# Minute distribution
if 'minute' in events.columns:
    print('\nEvents by minute (binned):')
    events['minute_bin'] = pd.cut(events['minute'], bins=[0, 15, 30, 45, 60, 75, 90, 120], 
                                   labels=['0-15', '16-30', '31-45', '46-60', '61-75', '76-90', '90+'])
    minute_bin_counts = events['minute_bin'].value_counts().sort_index()
    for bin_label, cnt in minute_bin_counts.items():
        pct = cnt/n_events*100
        print(f'  {bin_label:8s}: {cnt:8,} ({pct:5.2f}%)')
    
    # Late match events (injury time)
    late_events = events[events['minute'] > 90]
    print(f'\nEvents after 90 minutes: {len(late_events):,} ({len(late_events)/n_events*100:.2f}%)')
    if not late_events.empty:
        print(f'  Latest event at minute: {events["minute"].max():.0f}')

# Timestamp coverage
if 'timestamp' in events.columns:
    timestamp_missing = events['timestamp'].isna().sum()
    print(f'\nTimestamp missing: {timestamp_missing:,} ({timestamp_missing/n_events*100:.2f}%)')

# 5) Location Data Analysis
print('\n--- Location Data ---')
location_cols = ['location_x', 'location_y']
has_location = all(col in events.columns for col in location_cols)

if has_location:
    events_with_location = events[location_cols].notna().all(axis=1).sum()
    print(f'Events with location data: {events_with_location:,} ({events_with_location/n_events*100:.2f}%)')
    
    # Location statistics
    print('\nLocation X (field width):')
    print(f'  Min: {events["location_x"].min():.2f}')
    print(f'  Max: {events["location_x"].max():.2f}')
    print(f'  Mean: {events["location_x"].mean():.2f}')
    
    print('\nLocation Y (field length):')
    print(f'  Min: {events["location_y"].min():.2f}')
    print(f'  Max: {events["location_y"].max():.2f}')
    print(f'  Mean: {events["location_y"].mean():.2f}')
    
    # Field zones (rough approximation)
    events['zone_x'] = pd.cut(events['location_x'], bins=[0, 40, 80, 120], 
                               labels=['Defensive Third', 'Middle Third', 'Attacking Third'])
    zone_counts = events['zone_x'].value_counts()
    print('\nEvents by field zone (x-axis):')
    for zone, cnt in zone_counts.items():
        pct = cnt/n_events*100
        print(f'  {zone:20s}: {cnt:8,} ({pct:5.2f}%)')
else:
    print('No location columns found')

# 6) Key Event-Specific Metrics
print('\n--- Event-Specific Metrics ---')

# Shot analysis
if 'shot_statsbomb_xg' in events.columns:
    shots = events[events['type'] == 'Shot'].copy()
    print(f'Total shots: {len(shots):,}')
    
    if len(shots) > 0:
        print(f'\nxG statistics:')
        print(f'  Total xG: {shots["shot_statsbomb_xg"].sum():.2f}')
        print(f'  Avg xG per shot: {shots["shot_statsbomb_xg"].mean():.4f}')
        print(f'  Median xG: {shots["shot_statsbomb_xg"].median():.4f}')
        print(f'  Max xG: {shots["shot_statsbomb_xg"].max():.4f}')
        
        # High quality chances
        high_quality = shots[shots['shot_statsbomb_xg'] > 0.3]
        print(f'\nHigh quality chances (xG > 0.3): {len(high_quality):,} ({len(high_quality)/len(shots)*100:.2f}%)')
        
        # Shot outcomes
        if 'shot_outcome' in events.columns:
            shot_outcomes = shots['shot_outcome'].value_counts()
            print('\nShot outcomes:')
            for outcome, cnt in shot_outcomes.items():
                pct = cnt/len(shots)*100
                print(f'  {outcome:20s}: {cnt:5,} ({pct:5.2f}%)')

# Pass analysis
passes = events[events['type'] == 'Pass']
print(f'\nTotal passes: {len(passes):,}')

if len(passes) > 0 and 'pass_outcome' in events.columns:
    # Pass completion
    completed_passes = passes['pass_outcome'].isna().sum()  # NaN typically means completed
    print(f'  Completed passes: {completed_passes:,} ({completed_passes/len(passes)*100:.2f}%)')
    
    # Pass outcomes (for incomplete passes)
    incomplete_outcomes = passes['pass_outcome'].value_counts()
    if not incomplete_outcomes.empty:
        print('\n  Incomplete pass reasons:')
        for outcome, cnt in incomplete_outcomes.head(10).items():
            pct = cnt/len(passes)*100
            print(f'    {outcome:20s}: {cnt:5,} ({pct:5.2f}%)')
    
    # Pass types
    if 'pass_type' in events.columns:
        pass_types = passes['pass_type'].value_counts()
        print('\n  Pass types (top 10):')
        for ptype, cnt in pass_types.head(10).items():
            pct = cnt/len(passes)*100
            print(f'    {ptype:25s}: {cnt:6,} ({pct:5.2f}%)')

# Duel analysis
duels = events[events['type'] == 'Duel']
if len(duels) > 0 and 'duel_outcome' in events.columns:
    print(f'\nTotal duels: {len(duels):,}')
    duel_outcomes = duels['duel_outcome'].value_counts()
    print('  Duel outcomes:')
    for outcome, cnt in duel_outcomes.items():
        pct = cnt/len(duels)*100
        print(f'    {outcome:25s}: {cnt:5,} ({pct:5.2f}%)')

# 7) Context Flags
print('\n--- Context Flags ---')
flag_cols = ['under_pressure', 'off_camera', 'counterpress']
for flag_col in flag_cols:
    if flag_col in events.columns:
        flag_count = events[flag_col].sum() if events[flag_col].dtype == bool else (events[flag_col] == True).sum()
        pct = flag_count/n_events*100
        print(f'{flag_col:20s}: {flag_count:8,} events ({pct:5.2f}%)')

# 8) Play Patterns
if 'play_pattern' in events.columns:
    print('\n--- Play Patterns ---')
    play_patterns = events['play_pattern'].value_counts()
    for pattern, cnt in play_patterns.items():
        pct = cnt/n_events*100
        print(f'  {pattern:30s}: {cnt:8,} ({pct:5.2f}%)')

# 9) Position Analysis
if 'position' in events.columns:
    print('\n--- Player Positions ---')
    positions = events['position'].value_counts()
    print(f'Unique positions: {len(positions)}')
    print('\nTop 15 positions by event count:')
    for i, (pos, cnt) in enumerate(positions.head(15).items(), 1):
        pct = cnt/n_events*100
        print(f'{i:2d}. {pos:25s}: {cnt:7,} ({pct:5.2f}%)')

# 10) Data Quality Assessment
print('\n--- Data Quality ---')

# Missing values by column (top 20)
miss = events.isnull().sum()
miss_pct = (miss / len(events) * 100).round(2)
miss_df = pd.DataFrame({'count': miss, 'pct': miss_pct})
miss_df = miss_df[miss_df['count'] > 0].sort_values('count', ascending=False)

print(f'Columns with missing values: {len(miss_df)}/{len(events.columns)}')
print('\nTop 20 columns by missing data:')
for col, row in miss_df.head(20).iterrows():
    print(f'  {col:40s}: {row["count"]:10,.0f} ({row["pct"]:6.2f}%)')

# Duplicate events
if 'id' in events.columns:
    dup_id_count = events['id'].duplicated().sum()
    print(f'\nDuplicate event IDs: {dup_id_count:,}')
    if dup_id_count > 0:
        print('Warning: Duplicate event IDs detected!')

# Data type summary
print('\nData type summary:')
dtype_counts = events.dtypes.value_counts()
for dtype, cnt in dtype_counts.items():
    print(f'  {str(dtype):20s}: {cnt} columns')

# Index continuity check
if 'index' in events.columns:
    index_gaps = events.groupby('match_id')['index'].apply(
        lambda x: (x.sort_values().diff() > 1).sum()
    )
    matches_with_gaps = (index_gaps > 0).sum()
    if matches_with_gaps > 0:
        print(f'\n⚠ Matches with non-continuous event indices: {matches_with_gaps:,}')

print('\n' + '='*80)
print('Events deep EDA complete.')
print('='*80)


DEEP DIVE: events
Total events: 12,188,949
Total columns: 112

Unique matches: 3,464
Avg events per match: 3518.7

Events per match distribution:
  Min: 2,101
  25th percentile: 3,263
  Median: 3,503
  75th percentile: 3,767
  Max: 5,190

⚠ Matches with unusually few events (<2912): 173
  Top 3 examples:
    match_id 7444: 2862 events
    match_id 7456: 2726 events
    match_id 7483: 2565 events

--- Event Type Distribution ---
Unique event types: 35

Top 20 event types:
 1. Pass                     : 3,387,760 (27.79%) █████
 2. Ball Receipt*            : 3,167,310 (25.99%) █████
 3. Carry                    : 2,632,570 (21.60%) ████
 4. Pressure                 : 1,113,859 ( 9.14%) █
 5. Ball Recovery            :  366,673 ( 3.01%) 
 6. Duel                     :  257,861 ( 2.12%) 
 7. Clearance                :  158,993 ( 1.30%) 
 8. Block                    :  132,352 ( 1.09%) 
 9. Dribble                  :  122,047 ( 1.00%) 
10. Goal Keeper              :  106,574 ( 0.87%) 
11. 

In [ ]:
# Lineups: one row per player per match
lineups_path = DATA_DIR / "lineups.parquet"
if not lineups_path.exists():
    raise FileNotFoundError(f"Missing {lineups_path}. Make sure data is downloaded into data/Statsbomb")

lineups = pd.read_parquet(lineups_path)
# Show a compact preview to confirm successful load
print('Lineups shape:', lineups.shape)
lineups.head(3)

Lineups shape: (165820, 17)


,match_id,team_id,player_id,player_name,player_nickname,jersey_number,country_id,country_name,team_name,position_name,from_time,to_time,from_period,to_period,card_time,card_type,card_reason
0,15946,206,6839,Daniel Alejandro Torres Rojas,Dani Torres,16,49.0,Colombia,Deportivo Alavés,Center Defensive Midfield,00:00,67:48,1.0,2.0,09:15,Yellow Card,Foul Committed
1,15946,206,6855,Guillermo Alfonso Maripán Loaysa,Guillermo Maripán,6,45.0,Chile,Deportivo Alavés,Left Center Back,00:00,NaN,1.0,NaN,40:46,Yellow Card,Bad Behaviour
2,15956,217,5213,Gerard Piqué Bernabéu,Gerard Piqué,3,214.0,Spain,Barcelona,Right Center Back,00:00,NaN,1.0,NaN,59:01,Yellow Card,Foul Committed


In [ ]:
# --- Lineups: Focused Deep EDA ---
print('\n' + '='*80)
print('DEEP DIVE: lineups')
print('='*80)

# Helper function for minutes calculation 
def calculate_minutes(from_time, to_time):
    """
    Calculate minutes played from time strings like '00:00' and '45:30'
    Returns minutes as integer, or None if invalid
    """
    try:
        if pd.isna(to_time) or pd.isna(from_time):
            return None
        
        # Handle string conversion
        from_time_str = str(from_time).strip()
        to_time_str = str(to_time).strip()
        
        # Parse from_time
        from_parts = from_time_str.split(':')
        if len(from_parts) < 2:
            return None
        from_mins = int(from_parts[0]) + int(from_parts[1]) / 60.0
        
        # Parse to_time
        to_parts = to_time_str.split(':')
        if len(to_parts) < 2:
            return None
        to_mins = int(to_parts[0]) + int(to_parts[1]) / 60.0
        
        # Calculate difference
        minutes = to_mins - from_mins
        
        # Sanity check: should be between 0 and 120 (including extra time)
        if minutes < 0 or minutes > 130:
            return None
            
        return minutes
    except:
        return None

# ============================================================================
# SECTION 1: KEY TAKEAWAYS - DATASET OVERVIEW
# ============================================================================
print('[1] DATASET OVERVIEW - KEY TAKEAWAYS')
print('-' * 80)

n_lineups = len(lineups)
n_unique_matches = lineups['match_id'].nunique()
n_unique_players = lineups['player_id'].nunique()
n_unique_teams = lineups['team_id'].nunique()

print(f'\n SCALE:')
print(f'  • Total lineup entries: {n_lineups:,}')
print(f'  • Unique matches: {n_unique_matches:,}')
print(f'  • Unique players: {n_unique_players:,}')
print(f'  • Unique teams: {n_unique_teams:,}')
print(f'  • Avg players per match: {n_lineups/n_unique_matches:.1f}')

# Expected: ~22-25 players per match (11 starters × 2 teams + subs)
expected_avg = 22
if n_lineups/n_unique_matches < expected_avg:
    print(f'\n WARNING: Average is low (expected ~{expected_avg}+)')
    print(f'  • May indicate missing substitutes or incomplete lineups')

# Players per match distribution
players_per_match = lineups.groupby('match_id').size()
print(f'\n PLAYERS PER MATCH:')
print(f'  • Min: {players_per_match.min()}')
print(f'  • Median: {players_per_match.median():.0f}')
print(f'  • Max: {players_per_match.max()}')

# Flag unusual matches
low_player_threshold = 20  # Less than typical starting XI × 2
high_player_threshold = 30  # More than normal with full benches

low_player_matches = players_per_match[players_per_match < low_player_threshold]
high_player_matches = players_per_match[players_per_match > high_player_threshold]

if len(low_player_matches) > 0:
    print(f'\n Matches with <{low_player_threshold} players: {len(low_player_matches)}')
    print(f'  • Could indicate incomplete data or red cards')

if len(high_player_matches) > 0:
    print(f'\n  Matches with >{high_player_threshold} players: {len(high_player_matches)}')
    print(f'  • Normal for matches with many substitutes')

# ============================================================================
# SECTION 2: PLAYER & TEAM ANALYSIS
# ============================================================================
print('\n\n[2] PLAYER & TEAM COVERAGE')
print('-' * 80)

# Most frequent players
player_appearances = lineups['player_name'].value_counts()
print(f'\nMost frequent players (appearances):')
for i, (player, count) in enumerate(player_appearances.head(10).items(), 1):
    print(f'{i:2d}. {player:40s}: {count:3d} matches')

# Most frequent teams
team_appearances = lineups['team_name'].value_counts()
print(f'\nMost frequent teams (lineup entries):')
for i, (team, count) in enumerate(team_appearances.head(10).items(), 1):
    avg_per_match = count / (lineups[lineups['team_name'] == team]['match_id'].nunique())
    print(f'{i:2d}. {team:30s}: {count:5,} entries ({avg_per_match:.1f} players/match)')

# Players per team
players_per_team = lineups.groupby('team_name')['player_id'].nunique().sort_values(ascending=False)
print(f'\nTeams using most unique players:')
for i, (team, count) in enumerate(players_per_team.head(10).items(), 1):
    print(f'{i:2d}. {team:30s}: {count:3d} unique players')

# ============================================================================
# SECTION 3: POSITION ANALYSIS
# ============================================================================
print('\n\n[3] POSITION DISTRIBUTION')
print('-' * 80)

if 'position_name' in lineups.columns:
    position_counts = lineups['position_name'].value_counts()
    print(f'\nTotal positions: {len(position_counts)}')
    print(f'\nTop 15 positions:')
    for i, (pos, count) in enumerate(position_counts.head(15).items(), 1):
        pct = count/n_lineups*100
        print(f'{i:2d}. {pos:30s}: {count:6,} ({pct:5.2f}%)')
    
    # Expected distribution check
    expected_positions = ['Goalkeeper', 'Center Back', 'Full Back', 'Defensive Midfield', 
                         'Center Midfield', 'Forward', 'Wing']
    missing_positions = [pos for pos in expected_positions 
                        if not any(pos.lower() in p.lower() for p in position_counts.index)]
    
    if missing_positions:
        print(f'\n Expected positions not found: {", ".join(missing_positions)}')
else:
    print('\n No position_name column found')

# ============================================================================
# SECTION 4: SUBSTITUTION & PLAYING TIME ANALYSIS
# ============================================================================
print('\n\n[4] SUBSTITUTIONS & PLAYING TIME')
print('-' * 80)

if 'from_time' in lineups.columns and 'to_time' in lineups.columns:
    # Calculate playing time first
    print('Calculating playing time...')
    lineups['minutes_played'] = lineups.apply(
        lambda row: calculate_minutes(row['from_time'], row['to_time']), 
        axis=1
    )
    
    # Categorize players
    starters = lineups[lineups['from_time'] == '00:00'].copy()
    non_starters = lineups[lineups['from_time'] != '00:00'].copy()
    
    # Among non-starters, separate actual subs from unused bench
    actual_subs = non_starters[non_starters['minutes_played'].notna() & (non_starters['minutes_played'] > 0)]
    unused_bench = non_starters[non_starters['minutes_played'].isna() | (non_starters['minutes_played'] == 0)]
    
    # Also check starters who might have 0 minutes (red card before kickoff, injury in warmup, etc.)
    starters_who_played = starters[starters['minutes_played'].notna() & (starters['minutes_played'] > 0)]
    starters_unused = starters[starters['minutes_played'].isna() | (starters['minutes_played'] == 0)]
    
    print(f'\n PLAYER CATEGORIES:')
    print(f'  • Starters (listed at 00:00): {len(starters):,} ({len(starters)/n_lineups*100:.1f}%)')
    print(f'    - Actually played: {len(starters_who_played):,}')
    print(f'    - Did not play: {len(starters_unused):,}')
    print(f'  • Non-starters: {len(non_starters):,} ({len(non_starters)/n_lineups*100:.1f}%)')
    print(f'    - Substitutes (came on): {len(actual_subs):,}')
    print(f'    - Unused bench: {len(unused_bench):,}')
    
    print(f'\n PLAYERS WHO ACTUALLY PLAYED:')
    total_played = len(starters_who_played) + len(actual_subs)
    print(f'  • Total: {total_played:,} ({total_played/n_lineups*100:.1f}% of all entries)')
    print(f'  • Starters: {len(starters_who_played):,} ({len(starters_who_played)/total_played*100:.1f}%)')
    print(f'  • Substitutes: {len(actual_subs):,} ({len(actual_subs)/total_played*100:.1f}%)')
    
    # Substitutions per match (only actual subs)
    actual_subs_per_match = actual_subs.groupby('match_id').size()
    print(f'\n ACTUAL SUBSTITUTIONS PER MATCH:')
    print(f'  • Average: {actual_subs_per_match.mean():.1f}')
    print(f'  • Median: {actual_subs_per_match.median():.0f}')
    print(f'  • Max: {actual_subs_per_match.max():.0f}')
    print(f'  • Matches with subs: {len(actual_subs_per_match):,}/{n_unique_matches:,}')
    
    # Unused bench per match
    unused_per_match = unused_bench.groupby('match_id').size()
    print(f'\n UNUSED BENCH PLAYERS PER MATCH:')
    print(f'  • Average: {unused_per_match.mean():.1f}')
    print(f'  • Median: {unused_per_match.median():.0f}')
    print(f'  • Max: {unused_per_match.max():.0f}')
    
    # Playing time analysis - only for players who actually played
    valid_minutes = lineups['minutes_played'].dropna()
    valid_minutes = valid_minutes[valid_minutes > 0]  # Exclude 0 minutes
    
    if len(valid_minutes) > 0:
        print(f'\n PLAYING TIME STATISTICS ({len(valid_minutes):,} players who played):')
        print(f'  • Average: {valid_minutes.mean():.1f} minutes')
        print(f'  • Median: {valid_minutes.median():.1f} minutes')
        print(f'  • Min: {valid_minutes.min():.1f} minutes')
        print(f'  • Max: {valid_minutes.max():.1f} minutes')
        
        # Distribution
        print(f'\n  Playing time distribution:')
        full_90 = (valid_minutes >= 90).sum()
        min_60_90 = ((valid_minutes >= 60) & (valid_minutes < 90)).sum()
        min_30_60 = ((valid_minutes >= 30) & (valid_minutes < 60)).sum()
        min_15_30 = ((valid_minutes >= 15) & (valid_minutes < 30)).sum()
        min_under_15 = (valid_minutes < 15).sum()
        
        print(f'    - Full 90+ minutes:  {full_90:6,} ({full_90/len(valid_minutes)*100:5.1f}%)')
        print(f'    - 60-90 minutes:     {min_60_90:6,} ({min_60_90/len(valid_minutes)*100:5.1f}%)')
        print(f'    - 30-60 minutes:     {min_30_60:6,} ({min_30_60/len(valid_minutes)*100:5.1f}%)')
        print(f'    - 15-30 minutes:     {min_15_30:6,} ({min_15_30/len(valid_minutes)*100:5.1f}%)')
        print(f'    - <15 minutes:       {min_under_15:6,} ({min_under_15/len(valid_minutes)*100:5.1f}%)')
        
        # Separate stats for starters vs subs
        starter_minutes = starters_who_played['minutes_played'].dropna()
        sub_minutes = actual_subs['minutes_played'].dropna()
        
        if len(starter_minutes) > 0:
            print(f'\n Starters who played:')
            print(f'    - Average: {starter_minutes.mean():.1f} minutes')
            print(f'    - Median: {starter_minutes.median():.1f} minutes')
            print(f'    - Full 90+: {(starter_minutes >= 90).sum():,} ({(starter_minutes >= 90).sum()/len(starter_minutes)*100:.1f}%)')
        
        if len(sub_minutes) > 0:
            print(f'\n Substitutes who came on:')
            print(f'    - Average: {sub_minutes.mean():.1f} minutes')
            print(f'    - Median: {sub_minutes.median():.1f} minutes')
            print(f'    - <15 min (impact subs): {(sub_minutes < 15).sum():,} ({(sub_minutes < 15).sum()/len(sub_minutes)*100:.1f}%)')
        
        # Flag suspicious values
        suspicious = valid_minutes[valid_minutes > 120]
        if len(suspicious) > 0:
            print(f'\n  Suspicious: {len(suspicious)} entries with >120 minutes (possible extra time or data error)')
    else:
        print('\n No valid playing time data could be calculated')
        
    # Sample comparison
    print('\n SAMPLE DATA (first 3 of each category):')
    print('\nStarters who played:')
    display(starters_who_played[['player_name', 'team_name', 'from_time', 'to_time', 'minutes_played']].head(3))
    
    print('\nSubstitutes who came on:')
    display(actual_subs[['player_name', 'team_name', 'from_time', 'to_time', 'minutes_played']].head(3))
    
    print('\nUnused bench players:')
    display(unused_bench[['player_name', 'team_name', 'from_time', 'to_time', 'minutes_played']].head(3))
    
else:
    print('\n No from_time/to_time columns found for playing time analysis')


# ============================================================================
# SECTION 5: CARDS & DISCIPLINARY RECORDS
# ============================================================================
print('\n\n[5] DISCIPLINARY RECORDS')
print('-' * 80)

if 'card_type' in lineups.columns:
    cards = lineups[lineups['card_type'].notna()]
    print(f'\nTotal cards issued: {len(cards):,} ({len(cards)/n_lineups*100:.2f}% of players)')
    
    if len(cards) > 0:
        card_types = cards['card_type'].value_counts()
        print(f'\nCard distribution:')
        for card, count in card_types.items():
            pct = count/len(cards)*100
            print(f'  • {card:15s}: {count:5,} ({pct:5.2f}%)')
        
        # Card reasons
        if 'card_reason' in lineups.columns:
            card_reasons = cards['card_reason'].value_counts()
            print(f'\nTop card reasons:')
            for i, (reason, count) in enumerate(card_reasons.head(10).items(), 1):
                print(f'{i:2d}. {reason:30s}: {count:4,}')
        
        # Players with most cards
        players_with_cards = cards.groupby('player_name').size().sort_values(ascending=False)
        print(f'\nMost carded players:')
        for i, (player, count) in enumerate(players_with_cards.head(10).items(), 1):
            print(f'{i:2d}. {player:40s}: {count:2d} cards')
        
        # Teams with most cards
        teams_with_cards = cards.groupby('team_name').size().sort_values(ascending=False)
        print(f'\nMost carded teams:')
        for i, (team, count) in enumerate(teams_with_cards.head(10).items(), 1):
            cards_per_match = count / lineups[lineups['team_name'] == team]['match_id'].nunique()
            print(f'{i:2d}. {team:30s}: {count:3d} cards ({cards_per_match:.2f} per match)')
else:
    print('\n No card_type column found')

# ============================================================================
# SECTION 6: NATIONALITY ANALYSIS
# ============================================================================
print('\n\n[6] NATIONALITY DISTRIBUTION')
print('-' * 80)

if 'country_name' in lineups.columns:
    country_counts = lineups['country_name'].value_counts()
    print(f'\nTotal countries represented: {len(country_counts)}')
    print(f'\nTop 15 countries:')
    for i, (country, count) in enumerate(country_counts.head(15).items(), 1):
        pct = count/n_lineups*100
        unique_players = lineups[lineups['country_name'] == country]['player_id'].nunique()
        print(f'{i:2d}. {country:25s}: {count:5,} appearances ({unique_players:4,} players, {pct:5.2f}%)')
else:
    print('\n No country_name column found')

# ============================================================================
# SECTION 7: JERSEY NUMBERS
# ============================================================================
print('\n\n[7] JERSEY NUMBERS')
print('-' * 80)

if 'jersey_number' in lineups.columns:
    jersey_numbers = lineups['jersey_number'].dropna()
    print(f'\nJersey number coverage: {len(jersey_numbers):,}/{n_lineups:,} ({len(jersey_numbers)/n_lineups*100:.1f}%)')
    
    if len(jersey_numbers) > 0:
        print(f'\nJersey number range: {jersey_numbers.min():.0f} to {jersey_numbers.max():.0f}')
        
        # Most popular numbers
        popular_numbers = jersey_numbers.value_counts().head(10)
        print(f'\nMost popular jersey numbers:')
        for number, count in popular_numbers.items():
            print(f'  #{int(number):2d}: {count:5,} players')
        
        # Unusual numbers
        unusual = jersey_numbers[jersey_numbers > 50]
        if len(unusual) > 0:
            print(f'\nUnusual numbers (>50): {len(unusual):,} instances')
            print(f'  Highest: #{jersey_numbers.max():.0f}')
else:
    print('\n  No jersey_number column found')

# ============================================================================
# SECTION 8: DATA QUALITY
# ============================================================================
print('\n\n[8] DATA QUALITY')
print('-' * 80)

# Missing values
miss = lineups.isnull().sum()
miss_pct = (miss / len(lineups) * 100).round(2)
miss_df = pd.DataFrame({'count': miss, 'pct': miss_pct})
miss_df = miss_df[miss_df['count'] > 0].sort_values('count', ascending=False)

if miss_df.empty:
    print('\n✓ No missing values in any column')
else:
    print(f'\nColumns with missing values: {len(miss_df)}/{len(lineups.columns)}')
    print('\nTop 10 columns by missing data:')
    for col, row in miss_df.head(10).iterrows():
        print(f'  {col:30s}: {row["count"]:8,.0f} ({row["pct"]:6.2f}%)')

# Duplicates
if 'match_id' in lineups.columns and 'player_id' in lineups.columns:
    dup_count = lineups.duplicated(subset=['match_id', 'player_id']).sum()
    print(f'\nDuplicate (match_id, player_id): {dup_count:,}')
    if dup_count > 0:
        print(' Warning: Same player appears multiple times in same match!')
        dup_examples = lineups[lineups.duplicated(subset=['match_id', 'player_id'], keep=False)]
        print(f'  Examples:')
        display(dup_examples[["match_id", "player_name", "team_name"]].head(3))

# Data type summary
print('\nData type summary:')
dtype_counts = lineups.dtypes.value_counts()
for dtype, cnt in dtype_counts.items():
    print(f'  {str(dtype):20s}: {cnt} columns')

# Match coverage check
print(f'\n✓ Match coverage: {n_unique_matches:,} unique matches in lineups')

print('\n' + '='*80)
print('Lineups deep EDA complete.')
print('='*80)


DEEP DIVE: lineups
[1] DATASET OVERVIEW - KEY TAKEAWAYS
--------------------------------------------------------------------------------

 SCALE:
  • Total lineup entries: 165,820
  • Unique matches: 3,464
  • Unique players: 10,803
  • Unique teams: 312
  • Avg players per match: 47.9

 PLAYERS PER MATCH:
  • Min: 24
  • Median: 47
  • Max: 119

  Matches with >30 players: 3382
  • Normal for matches with many substitutes


[2] PLAYER & TEAM COVERAGE
--------------------------------------------------------------------------------

Most frequent players (appearances):
 1. Lionel Andrés Messi Cuccittini          : 832 matches
 2. Sergio Busquets i Burgos                : 520 matches
 3. Andrés Iniesta Luján                    : 481 matches
 4. Gerard Piqué Bernabéu                   : 426 matches
 5. Jordi Alba Ramos                        : 331 matches
 6. Xavier Hernández Creus                  : 328 matches
 7. Pedro Eliezer Rodríguez Ledesma         : 299 matches
 8. Daniel Alves d

,player_name,team_name,from_time,to_time,minutes_played
0,Daniel Alejandro Torres Rojas,Deportivo Alavés,00:00,67:48,67.80
3,Enes Ünal,Real Valladolid,00:00,69:51,69.85
8,Asier Illarramendi Andonegi,Real Sociedad,00:00,81:36,81.60



Substitutes who came on:


,player_name,team_name,from_time,to_time,minutes_played
4,Arturo Erasmo Vidal Pardo,Barcelona,70:46,70:49,0.050000
13,Clément Lenglet,Barcelona,36:01,45:00,8.983333
30,Sergio Busquets i Burgos,Barcelona,50:13,57:07,6.900000



Unused bench players:


,player_name,team_name,from_time,to_time,minutes_played
5,Arturo Erasmo Vidal Pardo,Barcelona,70:49,NaN,NaN
14,Clément Lenglet,Barcelona,45:00,NaN,NaN
23,Allan Romeo Nyom,Leganés,84:18,NaN,NaN




[5] DISCIPLINARY RECORDS
--------------------------------------------------------------------------------

Total cards issued: 19,606 (11.82% of players)

Card distribution:
  • Yellow Card    : 18,217 (92.92%)
  • Second Yellow  :   697 ( 3.56%)
  • Red Card       :   692 ( 3.53%)

Top card reasons:
 1. Foul Committed                : 16,118
 2. Bad Behaviour                 : 3,488

Most carded players:
 1. Sergio Busquets i Burgos                : 130 cards
 2. Gerard Piqué Bernabéu                   : 120 cards
 3. Lionel Andrés Messi Cuccittini          : 77 cards
 4. Daniel Alves da Silva                   : 74 cards
 5. Javier Alejandro Mascherano             : 73 cards
 6. Jordi Alba Ramos                        : 73 cards
 7. Neymar da Silva Santos Junior           : 48 cards
 8. Sergio Ramos García                     : 47 cards
 9. Rafael Márquez Álvarez                  : 44 cards
10. Andrés Iniesta Luján                    : 43 cards

Most carded teams:
 1. Barcelona    

,match_id,player_name,team_name
4,15973,Arturo Erasmo Vidal Pardo,Barcelona
5,15973,Arturo Erasmo Vidal Pardo,Barcelona
12,15986,Clément Lenglet,Barcelona



Data type summary:
  str                 : 10 columns
  int32               : 4 columns
  float64             : 4 columns

✓ Match coverage: 3,464 unique matches in lineups

Lineups deep EDA complete.


In [135]:
# Reference
reference_path = DATA_DIR / "reference.parquet"
if not reference_path.exists():
    raise FileNotFoundError(f"Missing {reference_path}. Make sure data is downloaded into data/Statsbomb")

reference = pd.read_parquet(reference_path)

In [136]:
# --- Reference: Deep EDA ---
print('\n' + '='*80)
print('DEEP DIVE: reference')
print('='*80)
print('One-liner: Reference tables for lookups (competitions, seasons, teams, players, etc.)\n')

# ============================================================================
# SECTION 1: DATASET OVERVIEW
# ============================================================================
print('[1] DATASET OVERVIEW')
print('-' * 80)

print(f'\nTotal rows in reference: {len(reference):,}')
print(f'Total columns: {len(reference.columns)}')

# Reference dataset uses 'table_name' to separate different lookup tables
if 'table_name' in reference.columns:
    unique_tables = reference['table_name'].unique()
    print(f'\nNumber of unique reference tables: {len(unique_tables)}')
    print(f'Tables: {", ".join(sorted(unique_tables))}')
    
    # Count by table
    table_counts = reference['table_name'].value_counts().sort_index()
    print(f'\nRows per table:')
    for table, count in table_counts.items():
        print(f'  • {table:20s}: {count:8,} rows')
    
    print('\n' + '='*80)
    print('BREAKDOWN BY REFERENCE TABLE')
    print('='*80)
    
    # Analyze each table separately
    for i, table_name in enumerate(sorted(unique_tables), 1):
        table_data = reference[reference['table_name'] == table_name].copy()
        
        print(f'\n{"="*70}')
        print(f'{i}. {table_name.upper()}')
        print(f'{"="*70}')
        
        print(f'Rows: {len(table_data):,}')
        
        # Get non-table_name columns (actual data columns)
        data_cols = [col for col in table_data.columns if col != 'table_name']
        print(f'Data columns: {len(data_cols)}')
        print(f'  {", ".join(data_cols)}')
        
        # Show sample data
        print(f'\nSample data (first 5 rows):')
        display(table_data[data_cols].head(5))
        
        # Show data shape for this table
        print(f'\nShape: {table_data[data_cols].shape}')
        
        # Check for key identifier columns
        potential_id_cols = [col for col in data_cols if 'id' in col.lower()]
        potential_name_cols = [col for col in data_cols if 'name' in col.lower()]
        
        if potential_id_cols or potential_name_cols:
            print(f'\nKey columns:')
            for id_col in potential_id_cols:
                unique_count = table_data[id_col].nunique()
                total_count = len(table_data)
                duplicates = total_count - unique_count
                print(f'  • {id_col}: {unique_count:,} unique (duplicates: {duplicates:,})')
            
            for name_col in potential_name_cols:
                unique_count = table_data[name_col].nunique()
                total_count = len(table_data)
                duplicates = total_count - unique_count
                print(f'  • {name_col}: {unique_count:,} unique (duplicates: {duplicates:,})')
        
        # Data quality for this table
        missing = table_data[data_cols].isnull().sum()
        missing = missing[missing > 0].sort_values(ascending=False)
        
        if not missing.empty:
            print(f'\nMissing data:')
            for col, count in missing.items():
                pct = count/len(table_data)*100
                print(f'  • {col:30s}: {count:6,} ({pct:5.1f}%)')
        else:
            print(f'\n✓ No missing data')
        
        # Data types
        print(f'\nData types:')
        dtypes = table_data[data_cols].dtypes.value_counts()
        for dtype, count in dtypes.items():
            print(f'  • {str(dtype):20s}: {count} columns')
        
        # Value counts for categorical-looking columns (if small number of unique values)
        for col in data_cols:
            if table_data[col].dtype == 'object' or table_data[col].dtype.name == 'category':
                unique_count = table_data[col].nunique()
                if unique_count <= 20:  # Only show if manageable number of categories
                    print(f'\nValue distribution for "{col}":')
                    value_counts = table_data[col].value_counts()
                    for val, count in value_counts.items():
                        pct = count/len(table_data)*100
                        print(f'  • {val:30s}: {count:5,} ({pct:5.1f}%)')
        
        print()

else:
    print('\n⚠️  No "table_name" column found!')
    print('\nAll columns:')
    for col in reference.columns:
        print(f'  • {col}')
    
    print('\nSample data (first 5 rows):')
    display(reference.head(5))

# ============================================================================
# SECTION 2: OVERALL DATA QUALITY SUMMARY
# ============================================================================
print('\n' + '='*80)
print('[2] OVERALL DATA QUALITY SUMMARY')
print('-' * 80)

# Total missing values
total_missing = reference.isnull().sum().sum()
total_cells = len(reference) * len(reference.columns)
print(f'\nTotal missing values: {total_missing:,} / {total_cells:,} cells ({total_missing/total_cells*100:.2f}%)')

# Duplicates check (if there's a table_name column)
if 'table_name' in reference.columns:
    print(f'\n✓ Reference data organized into {len(unique_tables)} separate lookup tables')
    
    # Check for duplicates within each table (excluding table_name)
    print(f'\nDuplicate row check per table:')
    for table_name in sorted(unique_tables):
        table_data = reference[reference['table_name'] == table_name].copy()
        data_cols = [col for col in table_data.columns if col != 'table_name']
        
        dup_count = table_data[data_cols].duplicated().sum()
        if dup_count > 0:
            print(f' {table_name}: {dup_count:,} duplicate rows')
        else:
            print(f' {table_name}: No duplicates')

# Data type summary
print('\nOverall data type distribution:')
dtype_counts = reference.dtypes.value_counts()
for dtype, count in dtype_counts.items():
    print(f'  • {str(dtype):20s}: {count} columns')

print('\n' + '='*80)
print('Reference deep EDA complete.')
print('='*80)


DEEP DIVE: reference
One-liner: Reference tables for lookups (competitions, seasons, teams, players, etc.)

[1] DATASET OVERVIEW
--------------------------------------------------------------------------------

Total rows in reference: 9,566
Total columns: 4

Number of unique reference tables: 6
Tables: country, event_type, play_pattern, player, position, team

Rows per table:
  • country             :      141 rows
  • event_type          :       35 rows
  • play_pattern        :        9 rows
  • player              :    9,043 rows
  • position            :       26 rows
  • team                :      312 rows

BREAKDOWN BY REFERENCE TABLE

1. COUNTRY
Rows: 141
Data columns: 3
  id, name, extra_info

Sample data (first 5 rows):


,id,name,extra_info
9425,154,Morocco,NaN
9426,160,Netherlands,NaN
9427,40,Canada,NaN
9428,91,Guadeloupe,NaN
9429,145,Mauritius,NaN



Shape: (141, 3)

Key columns:
  • id: 141 unique (duplicates: 0)
  • name: 141 unique (duplicates: 0)

Missing data:
  • extra_info                    :    141 (100.0%)

Data types:
  • str                 : 2 columns
  • int32               : 1 columns


2. EVENT_TYPE
Rows: 35
Data columns: 3
  id, name, extra_info

Sample data (first 5 rows):


,id,name,extra_info
9381,35,Starting XI,NaN
9382,42,Ball Receipt*,NaN
9383,16,Shot,NaN
9384,20,Own Goal Against,NaN
9385,2,Ball Recovery,NaN



Shape: (35, 3)

Key columns:
  • id: 35 unique (duplicates: 0)
  • name: 35 unique (duplicates: 0)

Missing data:
  • extra_info                    :     35 (100.0%)

Data types:
  • str                 : 2 columns
  • int32               : 1 columns


3. PLAY_PATTERN
Rows: 9
Data columns: 3
  id, name, extra_info

Sample data (first 5 rows):


...


Shape: (9043, 3)

Key columns:
  • id: 9,043 unique (duplicates: 0)
  • name: 9,038 unique (duplicates: 5)

Missing data:
  • extra_info                    :  9,043 (100.0%)

Data types:
  • str                 : 2 columns
  • int32               : 1 columns


5. POSITION
Rows: 26
Data columns: 3
  id, name, extra_info

Sample data (first 5 rows):


,id,name,extra_info
9355,25,Secondary Striker,NaN
9356,1,Goalkeeper,NaN
9357,9,Right Defensive Midfield,NaN
9358,24,Left Center Forward,NaN
9359,8,Left Wing Back,NaN



Shape: (26, 3)

Key columns:
  • id: 26 unique (duplicates: 0)
  • name: 26 unique (duplicates: 0)

Missing data:
  • extra_info                    :     26 (100.0%)

Data types:
  • str                 : 2 columns
  • int32               : 1 columns


6. TEAM
Rows: 312
Data columns: 3
  id, name, extra_info

Sample data (first 5 rows):


,id,name,extra_info
0,2021,Jamshedpur,male
1,7285,NorthEast United,male
2,865,England Women's,female
3,852,Norway Women's,female
4,1209,Chile Women's,female



Shape: (312, 3)

Key columns:
  • id: 312 unique (duplicates: 0)
  • name: 308 unique (duplicates: 4)

✓ No missing data

Data types:
  • str                 : 2 columns
  • int32               : 1 columns


[2] OVERALL DATA QUALITY SUMMARY
--------------------------------------------------------------------------------

Total missing values: 9,254 / 38,264 cells (24.18%)

✓ Reference data organized into 6 separate lookup tables

Duplicate row check per table:
 country: No duplicates
 event_type: No duplicates
 play_pattern: No duplicates
 player: No duplicates
 position: No duplicates
 team: No duplicates

Overall data type distribution:
  • str                 : 3 columns
  • int32               : 1 columns

Reference deep EDA complete.


In [141]:
# Three Sixty
three_sixty_path = DATA_DIR / "three_sixty.parquet"
if not three_sixty_path.exists():
    raise FileNotFoundError(f"Missing {three_sixty_path}. Make sure data is downloaded into data/Statsbomb")

three_sixty = pd.read_parquet(three_sixty_path)
three_sixty.head(3)

,match_id,id,event_uuid,teammate,actor,keeper,location_x,location_y,visible_area
0,3788742,174148,403db150-7e25-4850-aca5-aa7c54d0bbc3,False,False,False,86.038681,34.486008,"[85.9122658313659,78.7684484599881,54.39686712..."
1,3788742,174149,403db150-7e25-4850-aca5-aa7c54d0bbc3,True,False,False,89.149490,25.236403,"[85.9122658313659,78.7684484599881,54.39686712..."
2,3788742,174150,403db150-7e25-4850-aca5-aa7c54d0bbc3,False,False,False,89.975533,66.690712,"[85.9122658313659,78.7684484599881,54.39686712..."


In [144]:
# --- Three Sixty: Data Availability & Coverage Check ---
print('\n' + '='*80)
print('DEEP DIVE: three_sixty (360 tracking data)')
print('='*80)

# ============================================================================
# SECTION 1: BASIC OVERVIEW
# ============================================================================
print('[1] DATASET OVERVIEW')
print('-' * 80)

print(f'\nTotal 360 frames: {len(three_sixty):,}')
print(f'Total columns: {len(three_sixty.columns)}')
print(f'Columns: {", ".join(three_sixty.columns.tolist())}')

# ============================================================================
# SECTION 2: MATCH & EVENT COVERAGE
# ============================================================================
print('\n[2] MATCH & EVENT COVERAGE')
print('-' * 80)

# Match coverage
n_matches_with_360 = three_sixty['match_id'].nunique()
total_matches = matches['match_id'].nunique() if 'matches' in globals() else None

print(f'\nMatch coverage:')
print(f'  Matches with 360 data: {n_matches_with_360:,}')
if total_matches:
    pct = n_matches_with_360/total_matches*100
    print(f'  Total matches in dataset: {total_matches:,}')
    print(f'  Coverage percentage: {pct:.1f}%')
    print(f'  Matches without 360 data: {total_matches - n_matches_with_360:,}')

# Event coverage
n_events_with_360 = three_sixty['id'].nunique()
print(f'\nEvent coverage:')
print(f'  Unique events with 360 data: {n_events_with_360:,}')

# Frames per event distribution
frames_per_event = three_sixty.groupby('id').size()
print(f'\nFrames per event distribution:')
print(f'  Average: {frames_per_event.mean():.1f}')
print(f'  Median: {frames_per_event.median():.0f}')
print(f'  Min: {frames_per_event.min()}')
print(f'  Max: {frames_per_event.max()}')

# Show distribution
print(f'\nFrame count breakdown:')
print(f'  1 frame: {(frames_per_event == 1).sum():,} events')
print(f'  2-5 frames: {((frames_per_event >= 2) & (frames_per_event <= 5)).sum():,} events')
print(f'  6-10 frames: {((frames_per_event >= 6) & (frames_per_event <= 10)).sum():,} events')
print(f'  11-20 frames: {((frames_per_event >= 11) & (frames_per_event <= 20)).sum():,} events')
print(f'  20+ frames: {(frames_per_event > 20).sum():,} events')

# ============================================================================
# SECTION 3: MATCH-LEVEL COVERAGE ANALYSIS
# ============================================================================
print('\n[3] MATCH-LEVEL COVERAGE ANALYSIS')
print('-' * 80)

# Events per match with 360 data
events_per_match_360 = three_sixty.groupby('match_id')['id'].nunique()
print(f'\nEvents with 360 data per match:')
print(f'  Average: {events_per_match_360.mean():.1f}')
print(f'  Median: {events_per_match_360.median():.0f}')
print(f'  Min: {events_per_match_360.min()}')
print(f'  Max: {events_per_match_360.max()}')

# Compare to total events per match
if 'events' in globals():
    print(f'\nComparison to total events:')
    
    # Get events per match for matches that have 360 data
    matches_with_360 = three_sixty['match_id'].unique()
    events_in_360_matches = events[events['match_id'].isin(matches_with_360)]
    total_events_per_match = events_in_360_matches.groupby('match_id').size()
    
    # Join to see coverage
    coverage_df = pd.DataFrame({
        'total_events': total_events_per_match,
        'events_with_360': events_per_match_360
    }).fillna(0)
    coverage_df['coverage_pct'] = (coverage_df['events_with_360'] / coverage_df['total_events'] * 100).round(1)
    
    print(f'  Average events per match (all types): {coverage_df["total_events"].mean():.0f}')
    print(f'  Average events with 360 per match: {coverage_df["events_with_360"].mean():.0f}')
    print(f'  Average 360 coverage: {coverage_df["coverage_pct"].mean():.1f}%')
    
    print(f'\nCoverage distribution:')
    print(f'  <1% coverage: {(coverage_df["coverage_pct"] < 1).sum()} matches')
    print(f'  1-5% coverage: {((coverage_df["coverage_pct"] >= 1) & (coverage_df["coverage_pct"] < 5)).sum()} matches')
    print(f'  5-10% coverage: {((coverage_df["coverage_pct"] >= 5) & (coverage_df["coverage_pct"] < 10)).sum()} matches')
    print(f'  10-20% coverage: {((coverage_df["coverage_pct"] >= 10) & (coverage_df["coverage_pct"] < 20)).sum()} matches')
    print(f'  20%+ coverage: {(coverage_df["coverage_pct"] >= 20).sum()} matches')
    
    print(f'\nExamples:')
    print(f'  Highest coverage: {coverage_df["coverage_pct"].max():.1f}% (match_id: {coverage_df["coverage_pct"].idxmax()})')
    print(f'  Lowest coverage: {coverage_df["coverage_pct"].min():.1f}% (match_id: {coverage_df["coverage_pct"].idxmin()})')

# ============================================================================
# SECTION 4: FRAME ATTRIBUTES
# ============================================================================
print('\n[4] FRAME ATTRIBUTES')
print('-' * 80)

# Teammate/actor/keeper flags
print(f'\nPlayer role flags:')
if 'teammate' in three_sixty.columns:
    teammate_count = three_sixty['teammate'].sum()
    print(f'  Teammate frames: {teammate_count:,} ({teammate_count/len(three_sixty)*100:.1f}%)')

if 'actor' in three_sixty.columns:
    actor_count = three_sixty['actor'].sum()
    print(f'  Actor frames: {actor_count:,} ({actor_count/len(three_sixty)*100:.1f}%)')

if 'keeper' in three_sixty.columns:
    keeper_count = three_sixty['keeper'].sum()
    print(f'  Keeper frames: {keeper_count:,} ({keeper_count/len(three_sixty)*100:.1f}%)')

# Location data
print(f'\nLocation data:')
if 'location_x' in three_sixty.columns and 'location_y' in three_sixty.columns:
    has_location = three_sixty[['location_x', 'location_y']].notna().all(axis=1).sum()
    print(f'  Frames with location: {has_location:,} ({has_location/len(three_sixty)*100:.1f}%)')
    
    if has_location > 0:
        print(f'  Location X range: {three_sixty["location_x"].min():.1f} to {three_sixty["location_x"].max():.1f}')
        print(f'  Location Y range: {three_sixty["location_y"].min():.1f} to {three_sixty["location_y"].max():.1f}')

# Visible area
if 'visible_area' in three_sixty.columns:
    has_visible_area = three_sixty['visible_area'].notna().sum()
    print(f'\nVisible area data:')
    print(f'  Frames with visible area: {has_visible_area:,} ({has_visible_area/len(three_sixty)*100:.1f}%)')
    
    if has_visible_area > 0:
        sample_area = three_sixty[three_sixty['visible_area'].notna()]['visible_area'].iloc[0]
        print(f'  Data type: {type(sample_area).__name__}')
        if isinstance(sample_area, (list, str)):
            print(f'  Sample: {str(sample_area)[:100]}...')

# ============================================================================
# SECTION 5: WHICH MATCHES HAVE 360 DATA
# ============================================================================
print('\n[5] MATCHES WITH 360 DATA')
print('-' * 80)

if 'matches' in globals():
    matches_360_info = matches[matches['match_id'].isin(matches_with_360)].copy()
    
    print(f'\nCompetitions with 360 data:')
    comp_360 = matches_360_info['competition_name'].value_counts()
    for comp, count in comp_360.items():
        total_in_comp = len(matches[matches['competition_name'] == comp])
        pct = count/total_in_comp*100
        print(f'  {comp:40s}: {count:3,}/{total_in_comp:3,} matches ({pct:5.1f}%)')
    
    print(f'\nSeasons with 360 data:')
    season_360 = matches_360_info['season_name'].value_counts().sort_index()
    for season, count in season_360.head(10).items():
        print(f'  {season:20s}: {count:3,} matches')
    
    print(f'\nTeams with 360 data (top 10):')
    teams_360_home = matches_360_info['home_team'].value_counts()
    teams_360_away = matches_360_info['away_team'].value_counts()
    teams_360_total = (teams_360_home + teams_360_away).sort_values(ascending=False)
    
    for team, count in teams_360_total.head(10).items():
        print(f'  {team:30s}: {count:3,} matches')

# ============================================================================
# SECTION 6: DATA QUALITY
# ============================================================================
print('\n[6] DATA QUALITY')
print('-' * 80)

# Missing values
miss = three_sixty.isnull().sum()
miss_pct = (miss / len(three_sixty) * 100).round(2)
miss_df = pd.DataFrame({'count': miss, 'pct': miss_pct})
miss_df = miss_df[miss_df['count'] > 0].sort_values('count', ascending=False)

if miss_df.empty:
    print('\nNo missing values')
else:
    print(f'\nColumns with missing values:')
    for col, row in miss_df.items():
        print(f'  {col:30s}: {row["count"]:8,.0f} ({row["pct"]:6.2f}%)')

# Duplicates
dup_count = three_sixty.duplicated().sum()
print(f'\nDuplicate frames: {dup_count:,}')

# Data types
print(f'\nData type distribution:')
dtype_counts = three_sixty.dtypes.value_counts()
for dtype, count in dtype_counts.items():
    print(f'  {str(dtype):20s}: {count} columns')

print('\n' + '='*80)
print('Three Sixty deep EDA complete.')
print('='*80)


DEEP DIVE: three_sixty (360 tracking data)
[1] DATASET OVERVIEW
--------------------------------------------------------------------------------

Total 360 frames: 15,584,040
Total columns: 9
Columns: match_id, id, event_uuid, teammate, actor, keeper, location_x, location_y, visible_area

[2] MATCH & EVENT COVERAGE
--------------------------------------------------------------------------------

Match coverage:
  Matches with 360 data: 323
  Total matches in dataset: 3,464
  Coverage percentage: 9.3%
  Matches without 360 data: 3,141

Event coverage:
  Unique events with 360 data: 15,584,040

Frames per event distribution:
  Average: 1.0
  Median: 1
  Min: 1
  Max: 1

Frame count breakdown:
  1 frame: 15,584,040 events
  2-5 frames: 0 events
  6-10 frames: 0 events
  11-20 frames: 0 events
  20+ frames: 0 events

[3] MATCH-LEVEL COVERAGE ANALYSIS
--------------------------------------------------------------------------------

Events with 360 data per match:
  Average: 48247.8
  Media

# Exploratory Data Analysis Summary

## Dataset Overview

This analysis examines StatsBomb football event data comprising five primary datasets: matches, events, lineups, reference, and three_sixty. The data spans from 1958 to 2025, though coverage is highly concentrated in specific periods and competitions.

## Dataset Characteristics

### Matches Dataset
- **Size**: 3,464 matches across 21 competitions
- **Temporal Distribution**: Highly uneven coverage with 1,860 matches (54%) concentrated in 2015-2016, representing complete league seasons for top European competitions. Pre-2015 data consists of 425 scattered matches, primarily tournaments. Post-2016 coverage declines to 1,179 matches over 8 years.
- **Competition Focus**: La Liga dominates with 868 matches (25%), followed by Ligue 1 (435), Premier League (418), Serie A (381), and Bundesliga (340). Women's football represents 9.4% of matches.
- **Match Outcomes**: Home win rate of 45.2%, draw rate of 23.0%, and away win rate of 31.8%. Average goals per match: 2.85 (1.60 home, 1.26 away).
- **Data Quality**: No duplicate match IDs. Missing data primarily in non-essential columns (referee information, 360 availability flags).

### Events Dataset
- **Size**: 12,188,949 events across 3,464 matches
- **Event Distribution**: 
  - Pass: 3,387,760 (27.8%)
  - Ball Receipt: 3,167,310 (26.0%)
  - Carry: 2,632,570 (21.6%)
  - Pressure: 1,113,859 (9.1%)
  - Shot: 88,023 (0.7%)
- **Team Coverage**: Barcelona heavily overrepresented with 1,244,016 events (10.2%), reflecting the 2015-2016 peak coverage period. Top 10 players are all Barcelona 2010s era (Messi, Busquets, Xavi, Iniesta, etc.).
- **Shot Quality**: Total xG of 9,383 across 88,023 shots. Average xG per shot: 0.107. Only 8.1% of shots qualify as high-quality chances (xG > 0.3), suggesting dataset includes speculative efforts or reflects lower-quality chance creation.
- **Pass Completion**: 77.7% overall completion rate (2,633,436 of 3,387,760 passes completed).
- **Temporal Coverage**: Events distributed across periods with 50.4% in first half, 49.2% in second half, and 0.4% in extra time.
- **Data Quality**: No duplicate event IDs. Location data available for 99.25% of events. 20.9% of events occurred under pressure.

### Lineups Dataset
- **Size**: 165,820 lineup entries across 3,464 matches
- **Player Participation**: Only 53,883 entries (32.5%) represent players who actually played. The remaining 67.5% are squad listings without playing time, including 42,075 listed starters who never entered the match and 69,862 unused substitutes.
- **Substitution Patterns**: Average of 6.2 actual substitutions per match. Starters who played averaged 63.0 minutes; substitutes averaged 13.4 minutes.
- **Playing Time Distribution**: Among players who participated, only 2.1% played full 90+ minutes. 39.2% played 60-90 minutes, 22.8% played 30-60 minutes, and 25.2% played under 15 minutes.
- **Disciplinary Records**: 19,606 cards issued (11.8% of all lineup entries). Yellow cards account for 92.9% of cards. Sergio Busquets leads with 130 career cards in the dataset.
- **Geographic Distribution**: 141 countries represented. Spain leads with 24,180 appearances (14.6%), followed by France (9.1%) and England (8.5%).
- **Data Quality**: 33,919 duplicate (match_id, player_id) combinations (20.4% of dataset), primarily representing position changes or multiple stints in the same match rather than true errors.

### Reference Dataset
- **Size**: 9,566 rows organized into 6 lookup tables
- **Tables**: country (141), event_type (35), play_pattern (9), player (9,043), position (26), team (312)
- **Key Characteristics**: All ID columns are unique with no duplicates. Five duplicate player names and four duplicate team names exist, representing different entities with identical names.
- **Gender Information**: Team table includes gender classification (male/female) in the extra_info column, the only populated extra_info field across all reference tables.

### Three_Sixty Dataset
- **Size**: 15,584,040 frames representing player positions for specific events
- **Coverage**: Only 323 matches (9.3% of total dataset) have 360 data
- **Competition Coverage**: Tournament-focused with UEFA Euro at 100% coverage (102/102), UEFA Women's Euro at 95.2% (59/62), but only 10% coverage for Bundesliga (34/340). No 360 data for La Liga, Premier League, Serie A, or Ligue 1.
- **Temporal Coverage**: All 360 data from 2020-2025, with zero coverage during the dense 2015-2016 period.
- **Frame Attributes**: Each row represents one player's position at an event moment. 50% teammate frames, 6.6% actor frames, 2.3% goalkeeper frames. All frames include location coordinates and visible area polygons.

## Key Findings for Analysis

### Dataset Imbalances
The data exhibits strong Barcelona bias, with the club representing 10% of all events and the top 10 most frequent players all being 2010s Barcelona squad members. This concentration reflects the 2015-2016 coverage spike and limits generalizability to broader football populations.

### Temporal Concentration
The extreme concentration of data in 2015-2016 (54% of all matches in a 2-year window within a 67-year span) creates challenges for time-series analysis and requires careful consideration when drawing conclusions about football across different eras.

### Data Completeness Variability
While core event data shows excellent completeness (99.25% location coverage, zero duplicate events), peripheral datasets show significant gaps. The lineups dataset contains 67.5% non-participants, and 360 tracking data covers only 9.3% of matches.

### Shot Quality Patterns
The low percentage of high-quality chances (8.1% with xG > 0.3) and median xG of 0.055 per shot suggests either conservative shot selection in the dataset or inclusion of many low-probability efforts. This metric warrants careful interpretation in any xG-based analysis.

## Data Quality Assessment

**Strengths:**
- No duplicate identifiers in core datasets (matches, events)
- High location data completeness (99%+ in events)
- Complete timestamp coverage
- Comprehensive event type taxonomy (35 types)

**Limitations:**
- Severe temporal imbalance (54% of data in 2-year period)
- Team/player representation bias (Barcelona overweight)
- Lineups dataset contains 67.5% non-participants
- 360 data limited to 9.3% of matches
- Missing values concentrated in optional fields (cards, substitution details)

## Recommendations for Further Analysis

Given the dataset characteristics, analyses should account for:
1. Barcelona/La Liga overrepresentation when making general conclusions
2. Temporal clustering effects when performing time-based analyses
3. Limited 360 data availability when spatial analysis is required
4. Distinction between listed squad members and actual participants in lineups data
5. Tournament vs. league football differences in 360 data coverage